In [8]:
! git clone https://www.github.com/sterlinggutterman/Final-Project

fatal: destination path 'Final-Project' already exists and is not an empty directory.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [10]:
df = pd.read_parquet('Final-Project/full_df (2).parquet')
df.tail(40)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,reordered_yes/no,order_day,total_order_count,product_name,aisle_id,department_id,aisle,department
2419104,2056127,206105,prior,91,2,10,1.0,48775,3,1,yes,monday,99,Organic Red Cabbage,83,4,fresh vegetables,produce
2419105,2056127,206105,prior,91,2,10,1.0,14464,4,1,yes,monday,99,Dulce de Leche Caramel Ice Cream,37,1,ice cream ice,frozen
2419106,3149080,206105,prior,92,3,16,1.0,36695,1,0,no,tuesday,99,Organic Hothouse Cucumbers,83,4,fresh vegetables,produce
2419107,3149080,206105,prior,92,3,16,1.0,17794,2,1,yes,tuesday,99,Carrots,83,4,fresh vegetables,produce
2419108,3149080,206105,prior,92,3,16,1.0,12146,3,1,yes,tuesday,99,Duck Mousse with Port Wine,96,20,lunch meat,deli
2419109,3149080,206105,prior,92,3,16,1.0,18956,4,1,yes,tuesday,99,Truffle Mousse,67,20,fresh dips tapenades,deli
2419110,3149080,206105,prior,92,3,16,1.0,14335,5,1,yes,tuesday,99,Phish Food® Ice Cream,37,1,ice cream ice,frozen
2419111,3149080,206105,prior,92,3,16,1.0,42347,6,1,yes,tuesday,99,Caffeine Free Diet Cola All Natural Soda,77,7,soft drinks,beverages
2419112,3149080,206105,prior,92,3,16,1.0,22963,7,1,yes,tuesday,99,Organic Roasted Turkey Breast,96,20,lunch meat,deli
2419113,1347827,206105,prior,93,4,14,1.0,37876,1,0,no,wednesday,99,Grated Reggiano Parmigiano,21,16,packaged cheese,dairy eggs


In [11]:
df=df.drop(columns=["eval_set"])

In [12]:
import pickle
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [13]:
for each in df.columns:
    df[each]=df[each].fillna(0)

Lasso: Reordered

In [8]:
from sklearn import linear_model

# Target variable:
y = df['reordered']

# Numeric:
var_num = ['order_number', 'days_since_prior_order',
       'order_hour_of_day', 'order_dow', 'add_to_cart_order',
        'total_order_count']
X_num = df.loc[:,var_num]

# Dummies:
var_cat = ['order_day','department']
dummies = pd.DataFrame([])
for var in var_cat:
    new_dummies = pd.get_dummies( df.loc[:,var], dtype=int)
    dummies = pd.concat([dummies, new_dummies], axis=1)

# Create covariates:
X = pd.concat([X_num, dummies],axis=1)

# Scale data:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_sc = scaler.transform(X)

In [9]:
# Run LASSO:
reg = linear_model.Lasso(alpha=0.001, warm_start=True, max_iter=2000,fit_intercept=True)
reg.fit(X_sc,y)

sdf = pd.DataFrame({'variable': X.columns, 'slope':reg.coef_})
sdf


,variable,slope
0,order_number,0.093709
1,days_since_prior_order,0.022394
2,order_hour_of_day,-0.002593
3,order_dow,0.000000
4,add_to_cart_order,-0.029939
5,total_order_count,-0.002031
6,friday,-0.004111
7,monday,-0.000000
8,saturday,-0.001012
9,sunday,0.002046


Lasso: Add to Cart Order

In [14]:
from sklearn import linear_model

# Target variable:
y = df['add_to_cart_order']

# Numeric:
var_num = ['order_number', 'days_since_prior_order',
       'order_hour_of_day', 'order_dow', 'reordered',
        'total_order_count']
X_num = df.loc[:,var_num]

# Dummies:
var_cat = ['order_day','department']
dummies = pd.DataFrame([])
for var in var_cat:
    new_dummies = pd.get_dummies( df.loc[:,var], dtype=int)
    dummies = pd.concat([dummies, new_dummies], axis=1)

# Create covariates:
X = pd.concat([X_num, dummies],axis=1)

# Scale data:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_sc = scaler.transform(X)

In [15]:
# Run LASSO:
reg = linear_model.Lasso(alpha=0.01, warm_start=True, max_iter=2000,fit_intercept=True)
reg.fit(X,y)

sdf = pd.DataFrame({'variable': X.columns, 'slope':reg.coef_})
sdf

,variable,slope
0,order_number,0.004207
1,days_since_prior_order,0.235986
2,order_hour_of_day,-0.021183
3,order_dow,-0.054817
4,reordered,-1.211459
5,total_order_count,-0.001457
6,friday,0.000000
7,monday,-0.402037
8,saturday,0.000000
9,sunday,0.060852


Lasso: Days Since Prior Order

In [13]:
from sklearn import linear_model

# Target variable:
y = df['days_since_prior_order']

# Numeric:
var_num = ['order_number', 'add_to_cart_order',
       'order_hour_of_day', 'order_dow', 'reordered',
        'total_order_count']
X_num = df.loc[:,var_num]

# Dummies:
var_cat = ['order_day','department']
dummies = pd.DataFrame([])
for var in var_cat:
    new_dummies = pd.get_dummies( df.loc[:,var], dtype=int)
    dummies = pd.concat([dummies, new_dummies], axis=1)

# Create covariates:
X = pd.concat([X_num, dummies],axis=1)

# Scale data:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_sc = scaler.transform(X)

In [14]:
# Run LASSO:
reg = linear_model.Lasso(alpha=0.01, warm_start=True, max_iter=2000,fit_intercept=True)
reg.fit(X,y)

sdf = pd.DataFrame({'variable': X.columns, 'slope':reg.coef_})
sdf

,variable,slope
0,order_number,-0.005968
1,add_to_cart_order,0.050971
2,order_hour_of_day,-0.051592
3,order_dow,-0.097671
4,reordered,0.389086
5,total_order_count,-0.082274
6,friday,0.391592
7,monday,0.000000
8,saturday,-0.000000
9,sunday,0.099408


Lasso: Order Hour of Day

In [15]:
from sklearn import linear_model

# Target variable:
y = df['order_hour_of_day']

# Numeric:
var_num = ['order_number', 'add_to_cart_order',
       'days_since_prior_order', 'order_dow', 'reordered',
        'total_order_count']
X_num = df.loc[:,var_num]

# Dummies:
var_cat = ['order_day','department']
dummies = pd.DataFrame([])
for var in var_cat:
    new_dummies = pd.get_dummies( df.loc[:,var], dtype=int)
    dummies = pd.concat([dummies, new_dummies], axis=1)

# Create covariates:
X = pd.concat([X_num, dummies],axis=1)

# Scale data:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_sc = scaler.transform(X)

In [16]:
# Run LASSO:
reg = linear_model.Lasso(alpha=0.01, warm_start=True, max_iter=2000,fit_intercept=True)
reg.fit(X,y)

sdf = pd.DataFrame({'variable': X.columns, 'slope':reg.coef_})
sdf

,variable,slope
0,order_number,-0.002214
1,add_to_cart_order,-0.008448
2,days_since_prior_order,-0.101220
3,order_dow,0.018079
4,reordered,-0.048675
5,total_order_count,-0.019293
6,friday,0.080776
7,monday,-0.000000
8,saturday,0.466776
9,sunday,-0.294929


Lasso: Total Order Count

In [17]:
from sklearn import linear_model

# Target variable:
y = df['total_order_count']

# Numeric:
var_num = ['order_number', 'add_to_cart_order',
       'days_since_prior_order', 'order_dow', 'reordered',
        'order_hour_of_day']
X_num = df.loc[:,var_num]

# Dummies:
var_cat = ['order_day','department']
dummies = pd.DataFrame([])
for var in var_cat:
    new_dummies = pd.get_dummies( df.loc[:,var], dtype=int)
    dummies = pd.concat([dummies, new_dummies], axis=1)

# Create covariates:
X = pd.concat([X_num, dummies],axis=1)

# Scale data:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_sc = scaler.transform(X)

In [18]:
# Run LASSO:
reg = linear_model.Lasso(alpha=0.01, warm_start=True, max_iter=2000,fit_intercept=True)
reg.fit(X,y)

sdf = pd.DataFrame({'variable': X.columns, 'slope':reg.coef_})
sdf

,variable,slope
0,order_number,0.026502
1,add_to_cart_order,-0.002047
2,days_since_prior_order,-0.413606
3,order_dow,-0.000000
4,reordered,-0.042467
5,order_hour_of_day,-0.049600
6,friday,-0.199940
7,monday,0.063064
8,saturday,-0.240431
9,sunday,-0.000000
